In [13]:
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

PATH_TO_IMG = "IMG_20201210_100826.jpg"
PATH_TO_MODEL = "../RV2-NASNetMobile-non-trainable"


In [ ]:
# load model
model = load_model(PATH_TO_MODEL)


In [32]:
# load image and process
def get_proc(path):
    width = 75
    height = 150
    
    img = Image.open(path)
    img = img.resize((width, height), resample=1)
    img = img_to_array(img)
    img = img/255
    img = img.astype('float32')
    img = np.expand_dims(img, axis=0)
    
    return img
    
# prediction
def get_pred(img):
    return model.predict(img)[0,0]


# post_process
def post_process(pred):
    labels = ['good', 'kyphosis']
    threshold = 0.5
    
    if pred >= threshold:
        index = 1
        
    else:
        index = 0
        
    return labels[index]


def pipe(path):
    img = get_proc(path)
    pred = get_pred(img)
    label = post_process(pred)
    
    return label

In [34]:
pipe(PATH_TO_IMG)

'good'

In [ ]:
# def get_neck_exercise_and_stretch(type_of_exercise(freeweights and/or machine)):
    # request stretch
    # request exercise based on type_of_exercise


# def get_shoulders_exercise_and_stretch(type_of_exercise(freeweights and/or machine))
    # request stretch
    # request exercise based on type_of_exercise


    
    
    
# def main(type_of_exercise, part_of_body(neck, shoulders)):

        # if part_of_body == neck:
            # return get_neck_exercise_and_stretch(type_of_exercise)

        # elif part_of_body == shoulders:
            # return get_shoulders_exercise_and_stretch(type_of_exercise)
    
    
    
# if pred == kyphosis:
    # prompt user for part_of_body and type_of_exercise
    # run main()

In [36]:
np.random.choice(5, 5, replace=False)

array([0, 1, 3, 2, 4])